In [1]:
import requests
import pandas as pd

In [90]:
class NotProperResponseError(Exception):
    pass

class SomethingWrongError(Exception):
    pass

In [166]:
def get_history_hour(nbars=5, utc_to=None, coin=None, base_ccy=None, ex=None):    
    if utc_to is None:
        # 일봉 이하 봉차트의 시간: 봉의 시작점을 의미한다! 
        # 일봉과는 의미가 다르니 주의
        # 가장최근 시(hour)봉은 채워지고 있는 중이니, 전 시간대의 시봉을 가져온다
        utc_to = pd.Timestamp.utcnow() + pd.Timedelta(-1, unit='h')
        
    else:
        utc_to = pd.Timestamp(utc_to)
    
    baseurl = ('https://min-api.cryptocompare.com/data/histohour?'
               'fsym={fsym}'
               '&tsym={tsym}'
               '&e={e}'
               '&limit={limit}'
               '&toTs={toTs}')
    
    url = baseurl.format(fsym=coin, tsym=base_ccy, e=ex, limit=nbars, toTs=int(utc_to.timestamp()))
    resp = requests.get(url).json()
    msg = resp['Response']
    
    if msg == 'Success':
        return {pd.to_datetime(_r['time'], unit='s'):_r for _r in resp['Data'] if _r['close'] != 0}
    
    elif msg == 'Error':
        raise NotProperResponseError()
        
    else:
        raise SomethingWrongError()

In [167]:
get_history_hour(nbars=1, utc_to='2019-06-26 08:00', coin='BTC', base_ccy='KRW', ex='upbit')

{Timestamp('2019-06-26 07:00:00'): {'time': 1561532400,
  'close': 14983000,
  'high': 15322000,
  'low': 14983000,
  'open': 15316000,
  'volumefrom': 328.97,
  'volumeto': 4977177155.46},
 Timestamp('2019-06-26 08:00:00'): {'time': 1561536000,
  'close': 14877000,
  'high': 15000000,
  'low': 14853000,
  'open': 14983000,
  'volumefrom': 67.97,
  'volumeto': 1015067592.44}}

In [169]:
get_history_hour(nbars=1, utc_to='2019-06-26 08:00', coin='BTC', base_ccy='KRW', ex='upbit')

{Timestamp('2019-06-26 07:00:00'): {'time': 1561532400,
  'close': 14983000,
  'high': 15322000,
  'low': 14983000,
  'open': 15316000,
  'volumefrom': 328.97,
  'volumeto': 4977177155.46},
 Timestamp('2019-06-26 08:00:00'): {'time': 1561536000,
  'close': 15125000,
  'high': 15151000,
  'low': 14853000,
  'open': 14983000,
  'volumefrom': 225.36,
  'volumeto': 3375136770.46}}

In [96]:
# url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=1'
# url = 'https://api.coingecko.com/api/v3/exchanges/bithumb'
url = 'https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=KRW&limit=1&e=upbit'
resp = requests.get(url).json(); resp

{'Response': 'Success',
 'Type': 100,
 'Aggregated': False,
 'Data': [{'time': 1561525200,
   'close': 14951000,
   'high': 15592000,
   'low': 14930000,
   'open': 14939000,
   'volumefrom': 494.6,
   'volumeto': 7513933948.41},
  {'time': 1561528800,
   'close': 15119000,
   'high': 15153000,
   'low': 14951000,
   'open': 14951000,
   'volumefrom': 192.89,
   'volumeto': 2904418221.13}],
 'TimeTo': 1561528800,
 'TimeFrom': 1561525200,
 'FirstValueInArray': True,
 'ConversionType': {'type': 'force_direct', 'conversionSymbol': ''},
 'RateLimit': {},
 'HasWarning': False}

In [101]:
int((1561528800-1561525200)/3600)

1

In [75]:
prices = pd.DataFrame(resp['Data'])
prices

,close,high,low,open,time,volumefrom,volumeto
0,14286000,14410000,14178000,14304000,1561510800,367.01,5.249753e+09
1,14300000,14307000,14043000,14286000,1561514400,428.64,6.082490e+09
2,14697000,14699000,14279000,14300000,1561518000,423.25,6.105994e+09
3,14842000,14842000,14571000,14697000,1561521600,321.21,4.710727e+09


In [9]:
prices['time'] = pd.to_datetime(prices['time'], unit='s') + pd.Timedelta(9, unit='h'); prices

,close,high,low,open,time,volumefrom,volumeto
0,5676000,5712000,5661000,5680000,2019-04-04 04:00:00,206.51,1.174834e+09
1,5829000,5837000,5676000,5676000,2019-04-04 05:00:00,308.03,1.777735e+09
2,5753000,5863000,5753000,5829000,2019-04-04 06:00:00,376.73,2.193541e+09
3,5683000,5817000,5606000,5753000,2019-04-04 07:00:00,389.07,2.238989e+09
4,5599000,5700000,5543000,5683000,2019-04-04 08:00:00,349.13,1.956111e+09
5,5652000,5674000,5510000,5599000,2019-04-04 09:00:00,335.81,1.885486e+09
6,5637000,5718000,5622000,5652000,2019-04-04 10:00:00,407.94,2.317694e+09
7,5654000,5674000,5635000,5637000,2019-04-04 11:00:00,322.19,1.823208e+09
8,5568000,5664000,5560000,5654000,2019-04-04 12:00:00,359.71,2.018646e+09
9,5637000,5638000,5551000,5568000,2019-04-04 13:00:00,317.28,1.779670e+09


In [76]:
prices['time'].iloc[0]

1561510800

In [143]:
pd.to_datetime(1561532400, unit='s') + pd.Timedelta(9, unit='h')

Timestamp('2019-06-26 16:00:00')

In [153]:
pd.to_datetime(time(), unit='s')

Timestamp('2019-06-26 07:33:12.040388107')

In [154]:
pd.to_datetime(time(), unit='s', utc=True)

Timestamp('2019-06-26 07:33:21.380077362+0000', tz='UTC')

In [133]:
t = pd.Timestamp.utcnow()#.timestamp(); t
t

Timestamp('2019-06-26 07:06:56.221898+0000', tz='UTC')

In [91]:
pd.Timestamp('2000-01-01').timestamp()

946684800.0

In [112]:
('a'
'b')

'ab'

In [158]:
pd.Timestamp.utcnow() + pd.Timedelta(-1, unit='h')

Timestamp('2019-06-26 07:05:25.625152+0000', tz='UTC')